In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import sys
sys.path.append("..")
import os
import shutil
#from utils.model import Net_mask
#from utils.data import Data
#from scripts.parameters import get_parameters, get_parameters_1
from utils.model3 import Net_mask
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.distributions.multivariate_normal import MultivariateNormal

In [3]:
n_param = 10
number_of_data = 100_000
tensor_data = torch.zeros(number_of_data, 2*n_param)
tensor_d_sample =  MultivariateNormal(loc=torch.zeros(n_param), covariance_matrix=(torch.eye(n_param)*30)).sample(sample_shape=(number_of_data,))
tensor_n_masked = torch.randint(n_param, (number_of_data,))
tensor_masks = torch.rand(number_of_data, n_param).argsort(dim=1)
tensor_masks = (tensor_masks < tensor_n_masked.unsqueeze(1))*1
tensor_data[:,:n_param] = tensor_d_sample*tensor_masks
tensor_data[:,n_param:] = tensor_masks
#torch.save(tensor_data, '../data/100_000_data_var_30/100_000_data_var_30.pt')  # only run if need new data

In [4]:
sigma2_eps = 0.01
n_param = 10
number_of_data = 100_000
#noise_data =  MultivariateNormal(loc=torch.zeros(n_param), covariance_matrix=(torch.eye(n_param)*sigma2_eps)).sample(sample_shape=(number_of_data,))
#noise_data.size()
tau2 = 0.5**2

In [14]:
# Do not run this cell
model = Net_mask()
n_weights = sum(p.numel() for p in model.parameters())
path_to_output_folder = '../data/rml/'
if os.path.exists(path_to_output_folder) and os.path.isdir(path_to_output_folder):
    shutil.rmtree(path_to_output_folder)
for i in range(100):
    path = '../data/rml/' + str(i)
    os.makedirs(path)
    noise_data =  MultivariateNormal(loc=torch.zeros(n_param), covariance_matrix=(torch.eye(n_param)*sigma2_eps)).sample(sample_shape=(number_of_data,))
    noise_param =  MultivariateNormal(loc=torch.zeros(n_weights), covariance_matrix=(torch.eye(n_weights)*tau2)).sample(sample_shape=(1,))
    name_noise_data = path + '/' + f'noise_data_{i}' + '.pt'
    name_noise_param = path + '/' + f'noise_param_{i}' + '.pt'
    torch.save(noise_data, name_noise_data)
    torch.save(noise_param, name_noise_param)

In [12]:
model = Net_mask()
model.parameters()
sum(p.numel() for p in model.parameters())

13210

In [6]:
sigma2_eps = 0.1
data_noise_mean = torch.zeros(n_param)
data_noise_covariance = torch.eye(n_param)*sigma2_eps
data_noise =  MultivariateNormal(loc=data_noise_mean, covariance_matrix=data_noise_covariance).sample(sample_shape=(n_data,))
#torch.save(data_noise, '../data/run1/data_noise_n_100000_var_0_1.pt')

NameError: name 'n_data' is not defined

In [ ]:
model = Net_mask()
torch.save(model.state_dict(), '../data/run1/model_weights.pth')

In [ ]:
#tau2 = 0.5**2
tau2 = 0.5
n_theta = torch.cat([param.view(-1) for param in model.parameters()]).size()[0]
theta_noise_mean = torch.zeros(n_theta)
theta_noise_covariance = torch.eye(n_theta)*tau2
theta_noise = MultivariateNormal(loc=theta_noise_mean, covariance_matrix=theta_noise_covariance).sample(sample_shape=(n_theta,))[0]
torch.save(theta_noise, '../data/run1/theta_noise.pt')

In [ ]:
n_data = 5
n_param = 10
sigma2_data = 1
data_mean = torch.zeros(n_param)
data_covariance = torch.eye(n_param)*sigma2_data

tensor_data = torch.zeros(n_data, 2*n_param)
tensor_d_sample =  MultivariateNormal(loc=data_mean, covariance_matrix=data_covariance).sample(sample_shape=(n_data,))
tensor_n_masked = torch.randint(n_param, (n_data,))
tensor_masks = torch.rand(n_data, n_param).argsort(dim=1)
tensor_masks = (tensor_masks < tensor_n_masked.unsqueeze(1))*1
tensor_data[:,:n_param] = tensor_d_sample*tensor_masks
tensor_data[:,n_param:] = tensor_masks
tensor_data

tensor([[ 1.9577, -0.0000, -1.5375, -0.0000, -0.0000,  0.0827,  1.1485,  0.9240,
         -0.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  1.0000,
          1.0000,  1.0000,  0.0000,  0.0000],
        [ 0.2819,  0.2763, -0.6372,  0.0000,  0.0000,  1.3755,  0.0000,  0.7951,
         -0.3029, -0.0072,  1.0000,  1.0000,  1.0000,  0.0000,  0.0000,  1.0000,
          0.0000,  1.0000,  1.0000,  1.0000],
        [-1.6031,  2.3550,  1.3365,  0.0000,  0.7494,  1.1786, -0.4001, -1.7238,
          0.4032,  1.0135,  1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  1.0000,
          1.0000,  1.0000,  1.0000,  1.0000],
        [ 1.1441,  0.0000, -0.0000,  0.5468, -0.0000, -0.0000, -0.2267, -0.0000,
         -0.1454,  1.2918,  1.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,
          1.0000,  0.0000,  1.0000,  1.0000],
        [ 0.0000, -0.0000, -0.9138,  0.1288,  1.0785, -1.4407,  0.3798,  0.0000,
         -0.0000, -0.0000,  0.0000,  0.0000,  1.0000,  1.0000,  1.0000,  1.0000,
      

In [ ]:
idx = torch.randperm(tensor_data.nelement())
tensor_data = tensor_data.view(-1)[idx].view(tensor_data.size())
tensor_data

tensor([[ 1.3755, -1.4407,  1.0000,  0.1288, -1.6031,  1.3365,  1.0000, -0.0000,
          1.0000,  1.0000, -0.0000,  1.0000, -0.1454, -1.7238,  1.0000, -0.0000,
          0.9240,  1.1485,  1.0000,  0.7494],
        [-0.4001,  1.0000,  0.0000,  1.0000, -0.3029,  1.0000,  0.0000,  1.0000,
          1.0000,  0.3798,  0.0000, -0.0000,  0.0000, -0.0000,  1.0000, -0.6372,
          1.0000, -0.0000,  0.2763,  0.0000],
        [ 1.0000,  0.0000,  1.0000,  1.1786, -0.0000,  0.0000,  1.2918,  1.0000,
          1.0785,  2.3550,  1.0000,  1.0135,  1.0000,  0.0000,  1.0000,  0.4032,
          1.0000,  0.0000,  0.0000, -0.0000],
        [ 0.0000,  1.1441,  0.0000,  1.0000,  0.0000, -0.0000,  0.0000,  0.0000,
         -0.9138,  1.0000,  0.0000, -0.0072,  1.0000,  1.0000,  0.0000,  0.0000,
          0.2819,  0.0000,  0.0000,  1.9577],
        [-1.5375,  1.0000, -0.0000,  0.7951,  0.0000,  1.0000,  1.0000,  0.0000,
          0.0827,  0.0000,  0.0000,  0.0000, -0.2267, -0.0000,  1.0000,  1.0000,
      

In [ ]:
tensor_data[2]

tensor([1.0000, 0.0000, 1.0000, 1.1786, -0.0000, 0.0000, 1.2918, 1.0000, 1.0785,
        2.3550, 1.0000, 1.0135, 1.0000, 0.0000, 1.0000, 0.4032, 1.0000, 0.0000,
        0.0000, -0.0000])

In [ ]:
s = tensor_data.size(0)
idx = torch.randperm(s)
tensor_data[idx,:]

tensor([[-1.5375,  1.0000, -0.0000,  0.7951,  0.0000,  1.0000,  1.0000,  0.0000,
          0.0827,  0.0000,  0.0000,  0.0000, -0.2267, -0.0000,  1.0000,  1.0000,
          0.5468,  1.0000,  0.0000,  0.0000],
        [ 0.0000,  1.1441,  0.0000,  1.0000,  0.0000, -0.0000,  0.0000,  0.0000,
         -0.9138,  1.0000,  0.0000, -0.0072,  1.0000,  1.0000,  0.0000,  0.0000,
          0.2819,  0.0000,  0.0000,  1.9577],
        [ 1.3755, -1.4407,  1.0000,  0.1288, -1.6031,  1.3365,  1.0000, -0.0000,
          1.0000,  1.0000, -0.0000,  1.0000, -0.1454, -1.7238,  1.0000, -0.0000,
          0.9240,  1.1485,  1.0000,  0.7494],
        [ 1.0000,  0.0000,  1.0000,  1.1786, -0.0000,  0.0000,  1.2918,  1.0000,
          1.0785,  2.3550,  1.0000,  1.0135,  1.0000,  0.0000,  1.0000,  0.4032,
          1.0000,  0.0000,  0.0000, -0.0000],
        [-0.4001,  1.0000,  0.0000,  1.0000, -0.3029,  1.0000,  0.0000,  1.0000,
          1.0000,  0.3798,  0.0000, -0.0000,  0.0000, -0.0000,  1.0000, -0.6372,
      

In [17]:
a = torch.load('../data/rml/0/noise_data_0.pt')
plt.hist(a)

KeyboardInterrupt: 